## Neural Network

In [1]:
import torch
import torch.nn as nn #neural net
import torch.nn.functional as F #Functions such as relu
import torch.optim as optim #Parameter optimization functions
import numpy as np
from scipy.io import loadmat #Loading data

Define the network

In [2]:
class ANN(nn.Module):
    
    def __init__(self):
        #Define the network
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(400, 25)
        self.fc2 = nn.Linear(25,10)
        self.Sigmoid = nn.Sigmoid()
        self.Softmax = nn.Softmax(dim = -1)

    def forward(self, x):
        x = self.Sigmoid(self.fc1(x))
        x = self.Softmax(self.fc2(x))
        return(x)
    
    def loadWeight(self, weights):
        
        fc1 = torch.from_numpy(weights["Theta1"][:,:-1]).float()
        fc1Bias = torch.from_numpy(weights["Theta1"][:,0]).float()
        fc2 = torch.from_numpy(weights["Theta2"][:,:-1]).float()
        fc2Bias = torch.from_numpy(weights["Theta2"][:,0]).float()
        
        print(fc1.shape)
        print(fc1Bias)
        print(fc2Bias)
        
        param = list(self.parameters())
        param[0].data.sub_(fc1)
        param[1].data.sub_(fc1Bias)
        param[2].data.sub_(fc2)
        param[3].data.sub_(fc2Bias)

### Load the data

In [3]:
def loadData(name):
    return(loadmat(name))

In [4]:
data = loadData("ex3data1.mat")
print("Input shape: ", data["X"].shape)
print("Output shape: ", data["y"].shape)

Input shape:  (5000, 400)
Output shape:  (5000, 1)


In [ ]:
#Helper function to get X tensor at index
def getX(index):
    

In [ ]:
#Helper function to get y tensor at inde

### Train the Network

In [5]:
#Initialize the neural net
classifier = ANN()
print(classifier)
#Create the optim
optimizer = optim.SGD(classifier.parameters(), lr=0.01)

ANN(
  (fc1): Linear(in_features=400, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=10, bias=True)
  (Sigmoid): Sigmoid()
  (Softmax): Softmax()
)


In [7]:

#For each of the featureset.
for i in range (len(data["X"][:,1])):
    #Train with SGD for each set. 
    optimizer.zero_grad()   # zero the gradient
    #Extract a feature i
    features = torch.from_numpy(data["X"][i]).float()
    #Extract the correct value
    target = torch.zeros((10))
    target[data["y"][i]-1] = 1.0
    #Define the MSE loss.
    output = classifier(features)
    loss = nn.MSELoss()(output, target)
    print(loss)
    #Backprop the loss.
    loss.backward()
    #Updates our weights using optim SGD.
    optimizer.step()


tensor(0.0906, grad_fn=<MseLossBackward>)
tensor(0.0908, grad_fn=<MseLossBackward>)
tensor(0.0904, grad_fn=<MseLossBackward>)
tensor(0.0905, grad_fn=<MseLossBackward>)
tensor(0.0907, grad_fn=<MseLossBackward>)
tensor(0.0911, grad_fn=<MseLossBackward>)
tensor(0.0906, grad_fn=<MseLossBackward>)
tensor(0.0901, grad_fn=<MseLossBackward>)
tensor(0.0908, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0904, grad_fn=<MseLossBackward>)
tensor(0.0905, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0915, grad_fn=<MseLossBackward>)
tensor(0.0904, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0905, grad_fn=<MseLossBackward>)
tensor(0.0903, grad_fn=<MseLossBackward>)
tensor(0.0905, grad_fn=<MseLossBackward>)
tensor(0.0894, grad_fn=<MseLossBackward>)
tensor(0.0904, grad_fn=<MseLossBackward>)
tensor(0.0900, grad_fn=<MseLossBac

tensor(0.0817, grad_fn=<MseLossBackward>)
tensor(0.0812, grad_fn=<MseLossBackward>)
tensor(0.0810, grad_fn=<MseLossBackward>)
tensor(0.0814, grad_fn=<MseLossBackward>)
tensor(0.0811, grad_fn=<MseLossBackward>)
tensor(0.0806, grad_fn=<MseLossBackward>)
tensor(0.0805, grad_fn=<MseLossBackward>)
tensor(0.0816, grad_fn=<MseLossBackward>)
tensor(0.0812, grad_fn=<MseLossBackward>)
tensor(0.0820, grad_fn=<MseLossBackward>)
tensor(0.0806, grad_fn=<MseLossBackward>)
tensor(0.0805, grad_fn=<MseLossBackward>)
tensor(0.0806, grad_fn=<MseLossBackward>)
tensor(0.0805, grad_fn=<MseLossBackward>)
tensor(0.0802, grad_fn=<MseLossBackward>)
tensor(0.0804, grad_fn=<MseLossBackward>)
tensor(0.0799, grad_fn=<MseLossBackward>)
tensor(0.0808, grad_fn=<MseLossBackward>)
tensor(0.0803, grad_fn=<MseLossBackward>)
tensor(0.0806, grad_fn=<MseLossBackward>)
tensor(0.0814, grad_fn=<MseLossBackward>)
tensor(0.0811, grad_fn=<MseLossBackward>)
tensor(0.0809, grad_fn=<MseLossBackward>)
tensor(0.0797, grad_fn=<MseLossBac

tensor(0.0641, grad_fn=<MseLossBackward>)
tensor(0.0642, grad_fn=<MseLossBackward>)
tensor(0.0642, grad_fn=<MseLossBackward>)
tensor(0.0634, grad_fn=<MseLossBackward>)
tensor(0.0653, grad_fn=<MseLossBackward>)
tensor(0.0641, grad_fn=<MseLossBackward>)
tensor(0.0662, grad_fn=<MseLossBackward>)
tensor(0.0632, grad_fn=<MseLossBackward>)
tensor(0.0638, grad_fn=<MseLossBackward>)
tensor(0.0630, grad_fn=<MseLossBackward>)
tensor(0.0656, grad_fn=<MseLossBackward>)
tensor(0.0637, grad_fn=<MseLossBackward>)
tensor(0.0636, grad_fn=<MseLossBackward>)
tensor(0.0622, grad_fn=<MseLossBackward>)
tensor(0.0628, grad_fn=<MseLossBackward>)
tensor(0.0615, grad_fn=<MseLossBackward>)
tensor(0.0631, grad_fn=<MseLossBackward>)
tensor(0.0637, grad_fn=<MseLossBackward>)
tensor(0.0644, grad_fn=<MseLossBackward>)
tensor(0.0627, grad_fn=<MseLossBackward>)
tensor(0.0630, grad_fn=<MseLossBackward>)
tensor(0.0621, grad_fn=<MseLossBackward>)
tensor(0.1002, grad_fn=<MseLossBackward>)
tensor(0.0999, grad_fn=<MseLossBac

tensor(0.0944, grad_fn=<MseLossBackward>)
tensor(0.0946, grad_fn=<MseLossBackward>)
tensor(0.0945, grad_fn=<MseLossBackward>)
tensor(0.0944, grad_fn=<MseLossBackward>)
tensor(0.0942, grad_fn=<MseLossBackward>)
tensor(0.0937, grad_fn=<MseLossBackward>)
tensor(0.0943, grad_fn=<MseLossBackward>)
tensor(0.0945, grad_fn=<MseLossBackward>)
tensor(0.0946, grad_fn=<MseLossBackward>)
tensor(0.0945, grad_fn=<MseLossBackward>)
tensor(0.0941, grad_fn=<MseLossBackward>)
tensor(0.0946, grad_fn=<MseLossBackward>)
tensor(0.0940, grad_fn=<MseLossBackward>)
tensor(0.0941, grad_fn=<MseLossBackward>)
tensor(0.0938, grad_fn=<MseLossBackward>)
tensor(0.0941, grad_fn=<MseLossBackward>)
tensor(0.0940, grad_fn=<MseLossBackward>)
tensor(0.0942, grad_fn=<MseLossBackward>)
tensor(0.0941, grad_fn=<MseLossBackward>)
tensor(0.0937, grad_fn=<MseLossBackward>)
tensor(0.0940, grad_fn=<MseLossBackward>)
tensor(0.0939, grad_fn=<MseLossBackward>)
tensor(0.0938, grad_fn=<MseLossBackward>)
tensor(0.0938, grad_fn=<MseLossBac

tensor(0.0857, grad_fn=<MseLossBackward>)
tensor(0.0848, grad_fn=<MseLossBackward>)
tensor(0.0855, grad_fn=<MseLossBackward>)
tensor(0.0857, grad_fn=<MseLossBackward>)
tensor(0.0860, grad_fn=<MseLossBackward>)
tensor(0.0856, grad_fn=<MseLossBackward>)
tensor(0.0848, grad_fn=<MseLossBackward>)
tensor(0.0855, grad_fn=<MseLossBackward>)
tensor(0.0847, grad_fn=<MseLossBackward>)
tensor(0.0854, grad_fn=<MseLossBackward>)
tensor(0.0852, grad_fn=<MseLossBackward>)
tensor(0.0856, grad_fn=<MseLossBackward>)
tensor(0.0846, grad_fn=<MseLossBackward>)
tensor(0.0851, grad_fn=<MseLossBackward>)
tensor(0.0858, grad_fn=<MseLossBackward>)
tensor(0.0851, grad_fn=<MseLossBackward>)
tensor(0.0845, grad_fn=<MseLossBackward>)
tensor(0.0854, grad_fn=<MseLossBackward>)
tensor(0.0851, grad_fn=<MseLossBackward>)
tensor(0.0852, grad_fn=<MseLossBackward>)
tensor(0.0847, grad_fn=<MseLossBackward>)
tensor(0.0844, grad_fn=<MseLossBackward>)
tensor(0.0841, grad_fn=<MseLossBackward>)
tensor(0.0851, grad_fn=<MseLossBac

...

tensor(0.0868, grad_fn=<MseLossBackward>)
tensor(0.0878, grad_fn=<MseLossBackward>)
tensor(0.0869, grad_fn=<MseLossBackward>)
tensor(0.0874, grad_fn=<MseLossBackward>)
tensor(0.0870, grad_fn=<MseLossBackward>)
tensor(0.0868, grad_fn=<MseLossBackward>)
tensor(0.0865, grad_fn=<MseLossBackward>)
tensor(0.0871, grad_fn=<MseLossBackward>)
tensor(0.0870, grad_fn=<MseLossBackward>)
tensor(0.0865, grad_fn=<MseLossBackward>)
tensor(0.0868, grad_fn=<MseLossBackward>)
tensor(0.0864, grad_fn=<MseLossBackward>)
tensor(0.0865, grad_fn=<MseLossBackward>)
tensor(0.0868, grad_fn=<MseLossBackward>)
tensor(0.0865, grad_fn=<MseLossBackward>)
tensor(0.0869, grad_fn=<MseLossBackward>)
tensor(0.0863, grad_fn=<MseLossBackward>)
tensor(0.0870, grad_fn=<MseLossBackward>)
tensor(0.0868, grad_fn=<MseLossBackward>)
tensor(0.0862, grad_fn=<MseLossBackward>)
tensor(0.0859, grad_fn=<MseLossBackward>)
tensor(0.0865, grad_fn=<MseLossBackward>)
tensor(0.0856, grad_fn=<MseLossBackward>)
tensor(0.0868, grad_fn=<MseLossBac

tensor(0.1009, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.1003, grad_fn=<MseLossBackward>)
tensor(0.1001, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1003, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.1004, grad_fn=<MseLossBackward>)
tensor(0.1002, grad_fn=<MseLossBackward>)
tensor(0.1000, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1003, grad_fn=<MseLossBackward>)
tensor(0.1002, grad_fn=<MseLossBackward>)
tensor(0.1008, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1004, grad_fn=<MseLossBackward>)
tensor(0.1006, grad_fn=<MseLossBackward>)
tensor(0.1004, grad_fn=<MseLossBackward>)
tensor(0.1005, grad_fn=<MseLossBackward>)
tensor(0.1003, grad_fn=<MseLossBac

tensor(0.0981, grad_fn=<MseLossBackward>)
tensor(0.0977, grad_fn=<MseLossBackward>)
tensor(0.0978, grad_fn=<MseLossBackward>)
tensor(0.0981, grad_fn=<MseLossBackward>)
tensor(0.0979, grad_fn=<MseLossBackward>)
tensor(0.0981, grad_fn=<MseLossBackward>)
tensor(0.0983, grad_fn=<MseLossBackward>)
tensor(0.0979, grad_fn=<MseLossBackward>)
tensor(0.0980, grad_fn=<MseLossBackward>)
tensor(0.0977, grad_fn=<MseLossBackward>)
tensor(0.0980, grad_fn=<MseLossBackward>)
tensor(0.0980, grad_fn=<MseLossBackward>)
tensor(0.0977, grad_fn=<MseLossBackward>)
tensor(0.0982, grad_fn=<MseLossBackward>)
tensor(0.0978, grad_fn=<MseLossBackward>)
tensor(0.0975, grad_fn=<MseLossBackward>)
tensor(0.0978, grad_fn=<MseLossBackward>)
tensor(0.0977, grad_fn=<MseLossBackward>)
tensor(0.0978, grad_fn=<MseLossBackward>)
tensor(0.0982, grad_fn=<MseLossBackward>)
tensor(0.0976, grad_fn=<MseLossBackward>)
tensor(0.0979, grad_fn=<MseLossBackward>)
tensor(0.0973, grad_fn=<MseLossBackward>)
tensor(0.0980, grad_fn=<MseLossBac

tensor(0.0963, grad_fn=<MseLossBackward>)
tensor(0.0964, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0963, grad_fn=<MseLossBackward>)
tensor(0.0963, grad_fn=<MseLossBackward>)
tensor(0.0959, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0962, grad_fn=<MseLossBackward>)
tensor(0.0962, grad_fn=<MseLossBackward>)
tensor(0.0956, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0958, grad_fn=<MseLossBackward>)
tensor(0.0958, grad_fn=<MseLossBackward>)
tensor(0.0957, grad_fn=<MseLossBackward>)
tensor(0.0962, grad_fn=<MseLossBackward>)
tensor(0.0965, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0961, grad_fn=<MseLossBackward>)
tensor(0.0960, grad_fn=<MseLossBackward>)
tensor(0.0960, grad_fn=<MseLossBackward>)
tensor(0.0953, grad_fn=<MseLossBackward>)
tensor(0.0959, grad_fn=<MseLossBackward>)
tensor(0.0959, grad_fn=<MseLossBac

tensor(0.0913, grad_fn=<MseLossBackward>)
tensor(0.0915, grad_fn=<MseLossBackward>)
tensor(0.0917, grad_fn=<MseLossBackward>)
tensor(0.0916, grad_fn=<MseLossBackward>)
tensor(0.0910, grad_fn=<MseLossBackward>)
tensor(0.0917, grad_fn=<MseLossBackward>)
tensor(0.0909, grad_fn=<MseLossBackward>)
tensor(0.0914, grad_fn=<MseLossBackward>)
tensor(0.0909, grad_fn=<MseLossBackward>)
tensor(0.0908, grad_fn=<MseLossBackward>)
tensor(0.0908, grad_fn=<MseLossBackward>)
tensor(0.0915, grad_fn=<MseLossBackward>)
tensor(0.0913, grad_fn=<MseLossBackward>)
tensor(0.0913, grad_fn=<MseLossBackward>)
tensor(0.0913, grad_fn=<MseLossBackward>)
tensor(0.0908, grad_fn=<MseLossBackward>)
tensor(0.0912, grad_fn=<MseLossBackward>)
tensor(0.0909, grad_fn=<MseLossBackward>)
tensor(0.0910, grad_fn=<MseLossBackward>)
tensor(0.0911, grad_fn=<MseLossBackward>)
tensor(0.0912, grad_fn=<MseLossBackward>)
tensor(0.0910, grad_fn=<MseLossBackward>)
tensor(0.0910, grad_fn=<MseLossBackward>)
tensor(0.0906, grad_fn=<MseLossBac

In [8]:
#Update the weights.

### Accuracy of loaded weights

In [9]:
#Load some weights
weights = loadData("ex3weights.mat")
classifier.loadWeight(weights)

torch.Size([25, 400])
tensor([-0.0226, -0.0984,  0.1162, -0.2397, -0.7316, -0.5979,  0.1546, -0.0337,
        -0.4107,  0.0235,  0.2477,  0.2653,  0.0943,  0.2022, -0.2030,  0.1046,
         0.1489, -0.0379, -0.3320, -0.2977, -0.4842, -0.3898, -0.1832, -0.7021,
        -0.3509])
tensor([-0.7610, -0.6179, -0.6893, -0.6783, -0.5966, -0.8779, -0.5275, -0.7490,
        -0.6665, -0.4609])
